# Land Use in Spokane
## Areeba Usman & Rose Martin

--------

#### Import Packages and Install Census Data

In [1]:
import os
import zipfile
import rasterio
import requests
import shutil

import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from rasterio.features import shapes
from shapely.geometry import shape

In [ ]:
!pip install -q censusdata

## Land Cover Data

In [3]:
url = "https://earthobs3.arcgis.com/arcgis/rest/services/ESA_CCI_Land_Cover_Time_Series/ImageServer"

bbox = "-117.823629,47.259272,-117.039763,48.047877"

params = {
    "bbox": "-117.823629,47.259272,-117.039763,48.047877",  # Remove the extra value
    "bboxSR": "4326",
    "imageSR": "4326",
    "size": "4000,4000", 
    "format": "tiff",  
    "f": "json",
}

# Request the image
response = requests.get(url, params=params)

if response.status_code == 200:
    image_data = response.json()
    image_url = image_data.get("href", None)  # Extract image URL

    if image_url:
        print(f"Downloading image from: {image_url}")

        # Download the raster image
        image_response = requests.get(image_url)
        image_filename = "Data/land_use.tif"

        with open(image_filename, "wb") as file:
            file.write(image_response.content)

        print(f"Image saved")
    else:
        print("No image URL found in response.")
else:
    print(f"Failed to fetch data: {response.status_code}, Response: {response.text}")

No image URL found in response.


#### Opening file and projecting to WGS 84

In [ ]:
land_use_fn = "./Data/spokane_landcover.tif"

proj_fn = os.path.splitext(land_use_fn)[0]+'_utm_gdalwarp.tif'
dst_crs = 'EPSG:4326'

In [ ]:
!gdalwarp -srcnodata 0 -t_srs $dst_crs $land_use_fn $proj_fn

In [ ]:
land_use_da = rxr.open_rasterio(land_use_fn).squeeze()

In [ ]:
#land_use_da.plot.imshow(cmap='inferno')

## Radiation Data

In [2]:
# Define the ImageServer exportImage URL
server_url = "https://gis.earthdata.nasa.gov/image/rest/services/POWER/POWER_901_MONTHLY_RADIATION_UTC/ImageServer/exportImage"

# Define parameters for exporting an image
params = {
    "bbox": "-117.823629,47.259272,-117.039763,48.047877,53063",  # Bounding box covering Spokane County
    "bboxSR": "4326",
    "imageSR": "4326",
    "size": "4000,4000",  # Resolution of the image
    "format": "tiff",  # Use "png" if you prefer a simple image
    "f": "json",
}

# Request the image
response = requests.get(server_url, params=params)

if response.status_code == 200:
    image_data = response.json()
    image_url = image_data.get("href", None)  # Extract image URL

    if image_url:
        print(f"Downloading image from: {image_url}")

        # Download the raster image
        image_response = requests.get(image_url)
        image_filename = "Data/radiation.tif"

        with open(image_filename, "wb") as file:
            file.write(image_response.content)

        print(f"Image saved")
    else:
        print("No image URL found in response.")
else:
    print(f"Failed to fetch data: {response.status_code}, Response: {response.text}")

Image saved


In [4]:
radiation_fn = "Data/radiation.tif"
radiation_da = rxr.open_rasterio(radiation_fn)

In [5]:
data = radiation_da.squeeze()
mask = radiation_da.notnull()
transform = radiation_da.rio.transform()

results = shapes(radiation_da.values, mask=mask.values, transform=transform)

polygons = []
for geom, value in results:
    geom = shape(geom) 
    if geom.is_valid:  
        polygons.append(geom)

radiation_gdf = gpd.GeoDataFrame(geometry=polygons)

radiation_gdf.to_file("Data/output_radiation_polygons.geojson", driver="GeoJSON")

## Parks & Protected Lands GDF

In [ ]:
parks_gdf = gpd.read_file('./Data/Park_Boundaries.geojson')
#parks_gdf.head(3)

In [ ]:
hull = tract_geom_gdf.geometry.union_all()
parks_gdf.clip(hull);

### WA Load in

In [ ]:
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json'
states_gdf = gpd.read_file(states_url)
states_gdf.head()

## 32048 is the EPSG recommended for Spokane County
states_proj_gdf = states_gdf.to_crs(4326)
WA_idx = states_proj_gdf["NAME"].isin(["Washington"])
wa_state_gdf = states_proj_gdf[WA_idx]

In [ ]:
#Code taken from lab 5
def get_census_data(tables, state, county, year=2019):
    '''Download census data for a given state and county fips code.'''

    # Download the data
    data = censusdata.download('acs5', year,  # Use 2019 ACS 5-year estimates
                               censusdata.censusgeo([('state', state), ('county', county), ('tract', '*')]),
                               list(tables.keys()))

    # Rename the column
    data.rename(columns=tables, inplace=True)

    # Extract information from the first column
    data['Name'] = data.index.to_series().apply(lambda x: x.name)
    data['SummaryLevel'] = data.index.to_series().apply(lambda x: x.sumlevel())
    data['State'] = data.index.to_series().apply(lambda x: x.geo[0][1])
    data['County'] = data.index.to_series().apply(lambda x: x.geo[1][1])
    data['Tract'] = data.index.to_series().apply(lambda x: x.geo[2][1])
    data.reset_index(drop=True, inplace=True)
    data = data[['Tract','Name']+list(tables.values())].set_index('Tract')
    
    return data

def get_census_tract_geom(state_fips, county_fips):
    '''Download census tract geometries for a given state and county fips code, storing in /tmp and cleaning up after.'''

    temp_dir = "/tmp/census_tracts"
    zip_path = os.path.join(temp_dir, f'tl_2019_{state_fips}_tract.zip')

    # Ensure temp directory exists
    os.makedirs(temp_dir, exist_ok=True)

    # Download the file
    url = f'https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_{state_fips}_tract.zip'
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {url}")

    # Save ZIP file to temp directory
    with open(zip_path, "wb") as file:
        file.write(response.content)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(temp_dir)

    # Find the shapefile in extracted contents
    for file in os.listdir(temp_dir):
        if file.endswith(".shp"):
            shapefile_path = os.path.join(temp_dir, file)
            break

    # Read the shapefile into a GeoDataFrame
    tracts = gpd.read_file(shapefile_path)

    # Filter by county and set index
    tracts = tracts[tracts['COUNTYFP'] == county_fips]
    tracts = tracts.rename(columns={'TRACTCE': 'Tract'}).set_index('Tract')

    # Cleanup: Remove extracted files and ZIP file
    shutil.rmtree(temp_dir)

    return tracts[['geometry']]

In [ ]:
tables = {
'B19013_001E': 'MedianIncome',
'B01003_001E': 'TotalPopulation',
'B01002_001E': 'MedianAge',
'B17001_002E': 'PopulationBelowPovertyLevel',
'B02001_002E': 'PopulationWhiteAlone',
'B02001_003E': 'PopulationBlackAlone',
'B02001_004E': 'PopulationAmericanIndianAlaskaNativeAlone',
'B02001_005E': 'PopulationAsianAlone',
'B02001_006E': 'PopulationNativeHawaiianPacificIslanderAlone',
'B02001_007E': 'PopulationSomeOtherRaceAlone',
'B02001_008E': 'PopulationTwoOrMoreRaces',
'B03002_003E': 'PopulationNotHispanicWhiteAlone',
'B03003_003E': 'PopulationHispanic',
'B25064_001E': 'MedianGrossRent',
'B25077_001E': 'MedianHomeValue',
'B25035_001E': 'MedianYearStructureBuilt',
'B25001_001E': 'TotalHousingUnits',
'B25004_001E': 'TotalVacantHousingUnits',
'B25003_002E': 'OccupiedHousingUnitsOwnerOccupied',
'B25003_003E': 'OccupiedHousingUnitsRenterOccupied',
'B27001_005E': 'PopulationNoHealthInsuranceCoverage',
}

In [ ]:
import censusdata

state_fips = '53'  
county_fips = '063' 

census_df = get_census_data(tables, state_fips, county_fips)
tract_geom_gdf = get_census_tract_geom(state_fips, county_fips)

census_Spokane_gdf = gpd.GeoDataFrame(census_df.join(tract_geom_gdf))
census_Spokane_gdf =  census_Spokane_gdf.to_crs('epsg:32610')

In [ ]:
census_Spokane_gdf = census_Spokane_gdf.to_crs(parks_gdf.crs)
spokane_clipped_gdf = census_Spokane_gdf.overlay(parks_gdf, how='difference')

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

spokane_clipped_gdf.plot(ax=ax, color="green", edgecolor='black')

plt.show()